In [0]:
%run /Workspace/Users/vishal.krishnan@neudesic.com/Silver_to_Gold/utils

In [0]:
# dim_campaign

# Select required columns from each source table, rename to match final schema
web_campaigns = df_web.select("campaign_id", "campaign_name", "campaign_start_date", "campaign_end_date") \
    .withColumnRenamed("campaign_start_date", "start_date") \
    .withColumnRenamed("campaign_end_date", "end_date")

meta_campaigns = df_meta.select("campaign_id", "campaign_name", "campaign_start_date", "campaign_end_date") \
    .withColumnRenamed("campaign_start_date", "start_date") \
    .withColumnRenamed("campaign_end_date", "end_date")

youtube_campaigns = df_youtube.select("campaign_id", "campaign_name", "campaign_start_date", "campaign_end_date") \
    .withColumnRenamed("campaign_start_date", "start_date") \
    .withColumnRenamed("campaign_end_date", "end_date")

# Union all sources and deduplicate
all_campaigns = web_campaigns.unionByName(meta_campaigns).unionByName(youtube_campaigns).distinct()

# Cast and enrich for final schema
campaign_df = all_campaigns.select(
    col("campaign_id"),
    col("campaign_name"),
    col("start_date"),
    col("end_date"),
    lit("Conversion").alias("objective"),
    lit(75000.00).cast(DecimalType(12, 2)).alias("budget"),
    lit("video").alias("creative_type")
).dropna(subset=["campaign_id", "campaign_name", "start_date", "end_date"])

display(campaign_df)
# Upsert into Delta

upsert_to_delta(campaign_df, "dim_campaign", ["campaign_id"], zorder_cols=["campaign_id"])
